In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import lightning as L


import torch.nn.functional as F

from torch.optim import Adam # optim contains many optimizers. This time we're using Adam
from torch.distributions.uniform import Uniform # So we can initialize our tensors with a uniform distribution

import pandas as pd ## to create dataframes from graph input
import matplotlib.pyplot as plt ## matplotlib allows us to draw graphs.
import seaborn as sns ## seaborn makes it easier to draw nice-looking graphs.

import spacy as spacy
import string
from torch.nn.utils.rnn import pad_sequence
import json
import os
import random
# import torch.optim.lr_scheduler as lr_scheduler
import torch.optim.lr_scheduler as lr_scheduler



torch.autograd.set_detect_anomaly(True)

In [24]:
vocab = {
    }
with open('models/vocab.json', 'r') as file:
    vocab = json.load(file)
print(len(vocab))

def remove_punctuation(text):
    exclude = set(string.punctuation + "\t") - {"'"}
    translator = str.maketrans("", "", "".join(exclude))
    cleaned_text = text.translate(translator)
    return cleaned_text

new_inputs = []
new_labels = []


old_vocab_size = len(vocab)
print(old_vocab_size)

vocab_size = 0
tokens_size = 0

print("input length: ", tokens_size)
print("vocab size: ", vocab_size)

2354
2354
input length:  0
vocab size:  0


In [3]:

tokens_size = 0

file_path = "dataset.txt"
directory_path = 'new_dataset'
old_vocab_size = len(vocab)
print(old_vocab_size)
# Loop through each file in the directory and read it line by line
for entry in os.scandir(directory_path):
    if entry.is_file():
        print(f"Reading lines from {entry.name}:")
        
        with open(entry.path, 'r') as file:
            totalLine = 0
            for line in file:
                if line.strip() == '':
                    continue
                text = remove_punctuation(line)
                text = text.replace("\n", "")
                text = text.lower()

                nlp = spacy.load('en_core_web_sm')

                # Tokenize the string using spaCy
                tokens = [token.text for token in nlp(text)]
                tokens_size += len(tokens)
        
                for  index, item in enumerate(tokens):
                    if item not in vocab:
                        # Assign a new index to the token
                        vocab[item] = len(vocab)
                if '<EOS>' not in vocab:
                    vocab['<EOS>'] = len(vocab)
                totalLine += 1    
                print(totalLine)


vocab_size = len(vocab)

print("input length: ", tokens_size)
print("vocab size: ", vocab_size)

for entry in os.scandir(directory_path):
    if entry.is_file():
        print(f"Reading lines from {entry.name}:")

        with open(entry.path, 'r') as file:
            for line in file:
                if line.strip() == '':
                    continue
                text = remove_punctuation(line)
                text = text.replace("\n", "")
                text = text.lower()

                nlp = spacy.load('en_core_web_sm')

                # Tokenize the string using spaCy
                tokens = [token.text for token in nlp(text)]

                input_list = []
                label_list = []

                for i in range(len(tokens)-1):
                    
                        input_list.append(F.one_hot(torch.tensor(vocab[tokens[i]]), vocab_size).to(dtype=torch.float))
                        label_list.append(F.one_hot(torch.tensor(vocab[tokens[i+1]]), vocab_size).to(dtype=torch.float))

                input_list.append(F.one_hot(torch.tensor(vocab[tokens[len(tokens)-1]]), vocab_size).to(dtype=torch.float))
                label_list.append(F.one_hot(torch.tensor(vocab['<EOS>']), len(vocab)).to(dtype=torch.float))
                # print(input_list)
                input_stack = torch.stack(input_list)
                label_stack = torch.stack(label_list)
                new_inputs.append(input_stack)
                new_labels.append(label_stack)

inputs = pad_sequence(new_inputs, batch_first=True, padding_value=0)
labels = pad_sequence(new_labels, batch_first=True, padding_value=0)

print("inputs: ", inputs)
print("labels:", labels)

0
Reading lines from dataset.txt:
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
26

In [25]:
old_inputs = torch.load('models/inputs.pth')
old_labels = torch.load('models/labels.pth')
vocab_diff = vocab_size - old_vocab_size

padding = (0, vocab_diff)  # (before, after)

# Pad the tensor along the third dimension
old_inputs = F.pad(old_inputs, padding, value=0)
old_labels = F.pad(old_labels, padding, value=0)

# print(old_inputs[0][0])

In [9]:
converted_old_inputs = [torch.tensor(subarray) for subarray in old_inputs]
converted_old_labels = [torch.tensor(subarray) for subarray in old_labels]

# Determine the number of elements to select from the first list (70%)
num_elements_to_select = int(0.7 * len(converted_old_inputs))

# Randomly select indices to sample from the first list
selected_indices = random.sample(range(len(converted_old_inputs)), num_elements_to_select)

# Use the selected indices to get the sampled portion from the first list
sampled_old_inputs = [converted_old_inputs[i] for i in selected_indices]
sampled_old_labels = [converted_old_labels[i] for i in selected_indices]

# Concatenate the sampled portion with the second list
inputs = sampled_old_inputs + new_inputs
labels = sampled_old_labels + new_labels

inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
labels = pad_sequence(labels, batch_first=True, padding_value=0)


C:\Users\P.asadi\AppData\Local\Temp\ipykernel_19692\445032583.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  converted_old_inputs = [torch.tensor(subarray) for subarray in old_inputs]
C:\Users\P.asadi\AppData\Local\Temp\ipykernel_19692\445032583.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  converted_old_labels = [torch.tensor(subarray) for subarray in old_labels]


In [12]:
dataset = TensorDataset(inputs, labels)
# smapler = 
dataloader = DataLoader(dataset)
# print(dataset.__sizeof__)

<built-in method __sizeof__ of TensorDataset object at 0x0000021A18DCB590>


In [26]:
learning_rates = []


In [27]:
class DecoderOnlyTransformer(L.LightningModule):
    def __init__(self):
        super().__init__()

        L.seed_everything(seed=66)
        
        self.embedding_dim = 256
   
        self.input_to_wordembeding = nn.Linear(in_features=vocab_size, out_features=self.embedding_dim, bias=False)
        self.wordembeding_to_query = nn.Linear(in_features=self.embedding_dim, out_features=self.embedding_dim, bias=False)
        self.wordembeding_to_key = nn.Linear(in_features=self.embedding_dim, out_features=self.embedding_dim, bias=False)
        self.wordembeding_to_value = nn.Linear(in_features=self.embedding_dim, out_features=self.embedding_dim, bias=False)     
        self.selfattention_to_finalweights = nn.Linear(in_features=self.embedding_dim, out_features=vocab_size, bias=True)

        max_sequence_length = tokens_size
        
        self.positional_encoding = self.positional_encoding(max_sequence_length, self.embedding_dim)

        self.training_step_outputs = []
        self.query_list = []
        self.key_list = []
        self.value_list = []
        self.iteration_weights = []
        self.similarity = []
        self.result = []

    def forward(self, input):
        self.query_list = []
        self.key_list = []
        self.value_list = []
        self.iteration_weights = []
        self.similarity = []
        self.result = []
        # print("input: ", input)
        # print("query gradients: ", self.wordembeding_to_query.weight.grad)
        # print("key gradients: ", self.wordembeding_to_key.weight.grad)
        # print("value gradients: ", self.wordembeding_to_value.weight.grad)
        data = {
            "q1": self.wordembeding_to_query.weight.detach()[0].numpy(), # [0] = Weights to top activation function
            "q2": self.wordembeding_to_query.weight.detach()[1].numpy(), # [1] = Weights to bottom activation function
            "k1": self.wordembeding_to_key.weight.detach()[0].numpy(), # [0] = Weights to top activation function
            "k2": self.wordembeding_to_key.weight.detach()[1].numpy(), # [1] = Weights to bottom activation function
            "v1": self.wordembeding_to_value.weight.detach()[0].numpy(), # [1] = Weights to bottom activation function
            "v2": self.wordembeding_to_value.weight.detach()[1].numpy(), # [1] = Weights to bottom activation function
            # "token": ["what", "is", "statquest", "<EOS>", "awesome", "how", "dog", "nice"],
            # "input": ["input1", "input2", "input3", "input4", "input5", "input6", "input7", "input8"]
        }
        df = pd.DataFrame(data)
        # print(df)

        embedded_word_list = []
        positional_embedded_word_list = []
        weights_list = []
        attention_list = []
        residualConnection_list = []
        output_values_list = []

        valid_one_hot_list = list(filter(self.is_valid_one_hot, input[0]))
        
        for idx, item in enumerate(valid_one_hot_list):
            # print("item: ", item)

            embedded_word = self.input_to_wordembeding(item)
            embedded_word = F.relu(embedded_word)
            embedded_word_list.append(embedded_word)
            positional_embedded_word = embedded_word_list[idx] + self.positional_encoding[0][idx]
            positional_embedded_word_list.append(positional_embedded_word)

            q = self.wordembeding_to_query(positional_embedded_word_list[idx])
            k = self.wordembeding_to_key(positional_embedded_word_list[idx])
            v = self.wordembeding_to_value(positional_embedded_word_list[idx])

            self.query_list.append(q)
            self.key_list.append(k)
            self.value_list.append(v)

            # print("query list: ", self.query_list)
            # print("key list: ", self.key_list)
            # print("value list: ", self.value_list)
            result_list = [torch.matmul(q, key) for key in self.key_list]
            result_stack = torch.stack(result_list)

            weights = torch.softmax(result_stack.view(1,-1), dim=1)[0]
            weights_list.append(weights)
            
            # print("mutliply q*k result: ", result_stack)
            # print("weights: ", weights)
            # print("weights list: ", weights_list)
            
            attention = torch.zeros_like(positional_embedded_word, requires_grad=True)
            
            for i in range(idx + 1):
                attention = attention + weights_list[idx][i] * self.value_list[i]
            attention_list.append(attention)
            residualConnection = attention_list[idx] + positional_embedded_word_list[idx]
            # print("attention: ", attention)
            # print("positional_embedded: ", positional_embedded_word_list[idx])
            # print("residualConnection: ", residualConnection)
            residualConnection_list.append(residualConnection)
            output_values = self.selfattention_to_finalweights(residualConnection_list[idx])
            output_values_list.append(output_values)

        # print("input Start------------------------------------------------------------------------------------------------------------")
        # print("embedded word:", embedded_word_list)
        # print("positional:", positional_embedded_word_list)
        # print("query:", self.query_list)
        # print("key:", self.key_list)
        # print("value:", self.value_list)
        # print("weights:", weights_list)
        # print("attention:", attention_list)
        # print("residual:", residualConnection_list)
        # print("outputs:", output_values_list)
        # print("input End--------------------------------------------------------------------------------------------------------------")

        return torch.stack(output_values_list)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        # print("output: ", y_hat)
        # print("label: ", y)
        # result = torch.softmax(y_hat, dim=1)
        valid_one_hot_list = list(filter(self.is_valid_one_hot, y[0]))
        # print("result: ", result)
        # loss = nn.MSELoss()(result, torch.stack(valid_one_hot_list).float())
        loss = nn.CrossEntropyLoss()(y_hat, torch.stack(valid_one_hot_list).float())

        learning_rate = self.optimizer.param_groups[0]['lr']
        learning_rates.append(learning_rate)
        # criterion = nn.BCEWithLogitsLoss()
        # loss = criterion(result, torch.stack(valid_one_hot_list).float())
        loss.backward(retain_graph=True)
        self.training_step_outputs.append(loss)

        # self.clear_intermediate_values()
        self.optimizer.zero_grad()
        print("loss: ", loss)
        return loss

    def configure_optimizers(self):
        # self.optimizer = optim.Adam(self.parameters(), lr=0.01)
        self.optimizer = optim.Adam(self.parameters(), lr=10.0)
        # self.scheduler = lr_scheduler(self.optimizer, 'min', patience=5, factor=0.5, min_lr=0.000001 ,verbose=True)
        self.scheduler = lr_scheduler.LinearLR(self.optimizer, start_factor=0.1, end_factor=0.001, total_iters=30)
        # return self.optimizer
        return {
            'optimizer': self.optimizer,
            'lr_scheduler': {
                'scheduler': self.scheduler,
                'monitor': 'avg_train_loss',  # Adjust the monitor metric as needed
                'interval': 'epoch',
                'frequency': 1,
            }
        }
    def on_train_epoch_end(self):
        # print(self.training_step_outputs)
        # print(self.training_step_outputs.size)
        avg_loss = torch.stack([x for x in self.training_step_outputs]).mean()  # Calculate the average loss
        print("avg-train-loss", avg_loss) # log training average loss
        self.scheduler.step(avg_loss)  # Update the scheduler
    
    def positional_encoding(self, max_len, d_model):
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float32) * -(torch.log(torch.tensor(10000.0)) / d_model))
        
        # Calculate the sine and cosine components
        pos_enc = torch.zeros((max_len, d_model))
        pos_enc[:, 0::2] = torch.sin(position * div_term)
        pos_enc[:, 1::2] = torch.cos(position * div_term)
        
        return pos_enc.unsqueeze(0)
    
    def clear_intermediate_values(self):
    # Clear intermediate values to release memory
        del self.query_list[:]
        del self.key_list[:]
        del self.value_list[:]
        del self.iteration_weights[:]
        del self.similarity[:]
        del self.result[:]
    
    def is_valid_one_hot(self, encoded_vector):
        # Check if the vector is binary and has exactly one 1
        return all(bit in [0, 1] for bit in encoded_vector) and encoded_vector.tolist().count(1) == 1
    


In [28]:
model = DecoderOnlyTransformer()
learning_rates = []
# print("loss: ", loss)



Seed set to 93
c:\Users\P.asadi\AppData\Local\anaconda3\Lib\site-packages\torch\nn\init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [ ]:
data = {
            "i1": model.input_to_wordembeding.weight.detach()[0].numpy(), # [0] = Weights to top activation function
            "i2": model.input_to_wordembeding.weight.detach()[1].numpy(), # [1] = Weights to bottom activation function
            
            # "f2": model.selfattention_to_finalweights.weight.detach()[1].numpy(), # [1] = Weights to bottom activation function
            # "token": ["what", "is", "statquest", "<EOS>", "awesome", "how", "dog", "nice"],
            # "input": ["input1", "input2", "input3", "input4", "input5", "input6", "input7", "input8"]
        }
df = pd.DataFrame(data)
print(df)

In [ ]:
print(model.positional_encoding)

In [ ]:
checkpoint = torch.load('model_linear.chpt', map_location='cpu')
print(model.selfattention_to_finalweights.weight.shape)
print(model.selfattention_to_finalweights.bias.shape)

print(model.selfattention_to_finalweights.weight.shape)
for name, param in checkpoint['state_dict'].items():
    if name.startswith('input_to_wordembeding'):
        # Adjust the size of the weight matrix
        model.input_to_wordembeding.weight.data[:, :param.size(1)] = param
        # print(param.shape)
    elif name.startswith('selfattention_to_finalweights'):
        # Adjust the size of the weight matrix
        if 'bias' in name:
            # Adjust the size of the bias
            print("in if:", param.shape, name)
            model.selfattention_to_finalweights.bias.data[ : param.size(0)] = param
        elif 'weight' in name:
            # Adjust the size of the weight matrix
            print("in else:", param.shape, name)
            model.selfattention_to_finalweights.weight.data[: param.size(0), :] = param
    elif name.startswith('wordembeding_to_query'):
        model.wordembeding_to_query.weight.data = param
    elif name.startswith('wordembeding_to_key'):
        model.wordembeding_to_key.weight.data = param
    elif name.startswith('wordembeding_to_value'):
        model.wordembeding_to_value.weight.data = param
        
        # print(param.shape)
print(model.input_to_wordembeding.weight.shape)

print(model.selfattention_to_finalweights.weight.shape)
print(model.selfattention_to_finalweights.bias.shape)

        
data = {
    "w1": model.input_to_wordembeding.weight.detach()[0].numpy(), # [0] = Weights to top activation function
    "w2": model.input_to_wordembeding.weight.detach()[1].numpy(), # [1] = Weights to bottom activation function
    "w3": model.input_to_wordembeding.weight.detach()[2].numpy(), # [0] = Weights to top activation function
    "w4": model.input_to_wordembeding.weight.detach()[3].numpy(), # [1] = Weights to bottom activation function
    "w5": model.input_to_wordembeding.weight.detach()[4].numpy(), # [1] = Weights to bottom activation function
    "w6": model.input_to_wordembeding.weight.detach()[5].numpy(), # [1] = Weights to bottom activation function
    # "token": ["what", "is", "statquest", "<EOS>", "awesome", "how", "dog", "nice"],
    # "input": ["input1", "input2", "input3", "input4", "input5", "input6", "input7", "input8"]
}
df = pd.DataFrame(data)
df

In [10]:
trainer = L.Trainer(max_epochs=100)
trainer.fit(model, dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


NameError: name 'dataloader' is not defined

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("")
writer.add_graph(model, inputs)
writer.close()



In [15]:
saved_state_dict = torch.load('backups/model.pt')
model.load_state_dict(saved_state_dict)

model

DecoderOnlyTransformer(
  (input_to_wordembeding): Linear(in_features=2354, out_features=256, bias=False)
  (wordembeding_to_query): Linear(in_features=256, out_features=256, bias=False)
  (wordembeding_to_key): Linear(in_features=256, out_features=256, bias=False)
  (wordembeding_to_value): Linear(in_features=256, out_features=256, bias=False)
  (selfattention_to_finalweights): Linear(in_features=256, out_features=2354, bias=True)
)

In [22]:
text = "He got tired of being the devil's advocate and now agrees"
text = remove_punctuation(text)
text = text.lower()
# Load the spaCy English model
nlp = spacy.load('en_core_web_sm')

# Tokenize the string using spaCy
tokens = [token.text for token in nlp(text)]

print("Original String:", text)
print("Tokens:", tokens)


prompt_list = []
# print("vocab", vocab)
# print("prompt list:", prompt_list)
for i in range(len(tokens)):
    # print("token:", tokens[i], "token number:" , vocab[tokens[i]])
    token = F.one_hot(torch.tensor(vocab[tokens[i]]), len(vocab)).to(dtype=torch.float)
    # print("one hot", token)
    prompt_list.append(token)
    # print(tokens[i], end=" ")

# print(prompt_list)
prompt_stack = torch.stack(prompt_list)
prompt = torch.unsqueeze(prompt_stack, dim=0)

# print(prompt)


softmax = nn.Softmax(dim=0) ## dim=0 applies softmax to rows, dim=1 applies softmax to columns
reverse_vocab = {index: word for word, index in vocab.items()}


for i in range(5):
    result = model(prompt)
    # print(result)
    indices = torch.argmax(result, dim=1)
    prompt_list.append(F.one_hot(torch.tensor(indices[len(indices)-1]), len(vocab)).to(dtype=torch.float))
    prompt_stack = torch.stack(prompt_list)
    prompt = torch.unsqueeze(prompt_stack, dim=0)
    print(reverse_vocab[indices[len(indices)-1].item()], end= " ")

Original String: he got tired of being the devil's advocate and now agrees
Tokens: ['he', 'got', 'tired', 'of', 'being', 'the', 'devil', "'s", 'advocate', 'and', 'now', 'agrees']


C:\Users\P.asadi\AppData\Local\Temp\ipykernel_56328\4262483623.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prompt_list.append(F.one_hot(torch.tensor(indices[len(indices)-1]), len(vocab) + 4).to(dtype=torch.float))


are people reservations by upload 